In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

In [2]:
# Testing load graph from json
deep_reload(src)

filename = 'graph.json'

graph = src.graph.graph_from_json(filename)

In [19]:
n = graph._node[1000000000]

fun = lambda n: (n['rng'] >= .34) & (n['rng'] < .67)|(n['ev_network_clean']=='Depot')
fun(n)

True

In [22]:
n

{'status': 1,
 'name': 'UC Davis Depot',
 'address': '1590 Tilia St.',
 'city': 'Davis',
 'state': 'CA',
 'zip': 95616,
 'x': -121.7709234,
 'y': 38.54165851,
 'dcfc': 0.0,
 'ev_network_clean': 'Depot',
 'depot': 1000000000,
 'rng': 0.7621841426621033,
 'vehicle': ['Bolt', 'Lightning', 'IONIQ 5', 'Model 3']}

In [3]:
import json

filename = 'parameters_cec_router.json'

with open(filename, 'r') as file:

    parameters = json.load(file)

In [5]:
# Assigning nodes to depots based on Voronoi cells
deep_reload(src)

center_nodes = parameters['depot_nodes']
weight = parameters['voronoi_weight']

graph = src.router.assign_depot(graph, center_nodes, weight = weight)

In [6]:
# Assinging random number to each node for selection
deep_reload(src)

seed = parameters['rng_seed']

graph = src.router.assign_rng(graph, seed)

In [21]:
# Assinging vehicles to nodes
deep_reload(src)

vehicles = parameters['vehicles']

graph = src.router.assign_vehicle(graph, vehicles)

In [28]:
# Producing subgraphs for routing
deep_reload(src)

categories = {
    'vehicle': list(parameters['vehicles'].keys()),
    'depot': parameters['depot_nodes'],
}

subgraphs = src.router.produce_subgraphs(graph, categories)
# subgraphs

In [29]:
subgraphs

{('Bolt', 1000000000): <networkx.classes.graph.Graph at 0x7f5094040310>,
 ('Bolt', 2000000000): <networkx.classes.graph.Graph at 0x7f50940403d0>,
 ('Lightning', 1000000000): <networkx.classes.graph.Graph at 0x7f508c4b7d10>,
 ('Lightning', 2000000000): <networkx.classes.graph.Graph at 0x7f508c4cee90>,
 ('IONIQ 5', 1000000000): <networkx.classes.graph.Graph at 0x7f508bdca390>,
 ('IONIQ 5', 2000000000): <networkx.classes.graph.Graph at 0x7f508b60fb10>,
 ('Model 3', 1000000000): <networkx.classes.graph.Graph at 0x7f508c4b7cd0>,
 ('Model 3', 2000000000): <networkx.classes.graph.Graph at 0x7f5089e14590>}

In [30]:
# Producing adjacency matrices for routing
deep_reload(src)

weights = parameters['route_weights']

adjacency = src.router.produce_adjacency(subgraphs, weights)

In [31]:
# Producing assignments or routing
deep_reload(src)

assignments = src.router.produce_assignments(subgraphs)

In [60]:
# Producing bounds
deep_reload(src)

vehicles = parameters['vehicles']
weights = parameters['route_weights']

route_bounds, leg_bounds, stop_weights = src.router.produce_bounds(
    subgraphs, vehicles, weights)

In [143]:
# Producing un-optimized routes
deep_reload(src)

combination = ('Bolt', 1000000000)
case_index = 1
route_bounds_case = [rb[case_index] for rb in route_bounds[combination]]
leg_bounds_case = [rb[case_index] for rb in leg_bounds[combination]]
stop_weights_case = [rb[case_index] for rb in stop_weights[combination]]

routes, success = src.router.clarke_wright(
    adjacency[combination],
    assignments[combination]['node_to_idx'][combination[1]],
    route_bounds_case,
    leg_bounds_case,
    stop_weights_case,
)

In [144]:
routes[:10]

[[213, 0, 213],
 [213, 1, 106, 31, 213],
 [213, 3, 25, 67, 142, 115, 174, 213],
 [213, 4, 27, 64, 10, 147, 77, 127, 112, 213],
 [213, 5, 41, 117, 151, 110, 213],
 [213, 7, 26, 45, 109, 22, 89, 213],
 [213, 9, 105, 111, 198, 185, 170, 213],
 [213, 11, 113, 44, 102, 175, 71, 98, 119, 213],
 [213, 12, 213],
 [213, 14, 128, 8, 210, 159, 125, 213]]

In [147]:
deep_reload(src)

weights, validity = src.router.evaluate_route(
    adjacency[combination],
    routes[1],
    route_bounds_case,
    leg_bounds_case,
    stop_weights_case,
)

weights[0]/1000, weights[1]/3600, validity

(388.9, 5.163081422050176, [True, True])

In [151]:
deep_reload(src)

weights, validity = src.router.evaluate_routes(
    adjacency[combination],
    routes,
    route_bounds_case,
    leg_bounds_case,
    stop_weights_case,
)

weights, validity

([[324460.0],
  [388900.0],
  [380210.0],
  [367260.0],
  [388960.0],
  [396320.0],
  [248870.0],
  [387510.0],
  [0.0],
  [400000.0],
  [422980.0],
  [259610.0],
  [397170.0],
  [556020.0],
  [396820.0],
  [390200.0],
  [399990.0],
  [399690.0],
  [179650.0],
  [464740.0],
  [322410.0],
  [395400.0],
  [385100.0],
  [579840.0],
  [398890.0],
  [390910.0],
  [574300.0],
  [384830.0],
  [433460.0],
  [396810.0],
  [370280.0],
  [398430.0],
  [354840.0],
  [580680.0],
  [389450.0],
  [395690.0],
  [572620.0],
  [511660.0],
  [398260.0],
  [580840.0],
  [575260.0],
  [398450.0],
  [384410.0],
  [0.0],
  [322510.0],
  [0.0],
  [384270.0],
  [89660.0],
  [523180.0],
  [511180.0],
  [391360.0],
  [0.0],
  [587220.0],
  [0.0],
  [394620.0],
  [506920.0],
  [93900.0],
  [592460.0],
  [351420.0],
  [0.0],
  [359890.0],
  [29860.0],
  [390500.0],
  [396130.0],
  [412700.0],
  [522040.0],
  [0.0],
  [0.0],
  [417000.0],
  [0.0]],
 [[True],
  [True],
  [True],
  [True],
  [True],
  [True],
  [True

In [120]:
leg_bounds_case, route_bounds_case

([[20000.0, 300000.0], [0, 28800]], [[0, 400000.0], [0, 28800]])

In [55]:
route_bounds

{('Bolt', 1000000000): [[[0, 400000.0], [0, 400000.0]],
  [[0, 10800], [0, 28800]]],
 ('Bolt', 2000000000): [[[0, 400000.0], [0, 400000.0]],
  [[0, 10800], [0, 28800]]],
 ('Lightning', 1000000000): [[[0, 400000.0], [0, 400000.0]],
  [[0, 10800], [0, 28800]]],
 ('Lightning', 2000000000): [[[0, 400000.0], [0, 400000.0]],
  [[0, 10800], [0, 28800]]],
 ('IONIQ 5', 1000000000): [[[0, 350000.0], [0, 350000.0]],
  [[0, 10800], [0, 28800]]],
 ('IONIQ 5', 2000000000): [[[0, 350000.0], [0, 350000.0]],
  [[0, 10800], [0, 28800]]],
 ('Model 3', 1000000000): [[[0, 500000.0], [0, 500000.0]],
  [[0, 10800], [0, 28800]]],
 ('Model 3', 2000000000): [[[0, 500000.0], [0, 500000.0]],
  [[0, 10800], [0, 28800]]]}